# Spark Bank Marketing Cleansing
This notebook will be used for experiment purpose to cleanse the data

## Import Libs

In [15]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

In [2]:
# Initate spark session
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("sparkxperiment") \
    .getOrCreate()

22/12/18 05:02:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

## Load Data

In [4]:
# See current dir
!pwd

/home/jovyan/work/notebooks


In [17]:
df_bank_marketing = (
    spark.read
    .format("csv")
    .option("sep", ";")
    .option("header", True)
    .load("/home/jovyan/work/data/bank-additional-full.csv")
)

In [18]:
df_bank_marketing.show()

+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
| 57|   services| married|        high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.

In [19]:
df_bank_marketing.count()

41188

In [20]:
df_bank_marketing.createOrReplaceTempView('bank_marketing')

## Format Standarization

In [21]:
# Rename column with dots, because spark cant read them
df_transform1 = df_bank_marketing.withColumnRenamed('emp.var.rate', 'emp_var_rate') \
    .withColumnRenamed('cons.price.idx', 'cons_price_idx') \
    .withColumnRenamed('cons.conf.idx', 'cons_conf_idx') \
    .withColumnRenamed('nr.employed', 'nr_employed')

In [22]:
df_transform1.printSchema()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp_var_rate: string (nullable = true)
 |-- cons_price_idx: string (nullable = true)
 |-- cons_conf_idx: string (nullable = true)
 |-- euribor3m: string (nullable = true)
 |-- nr_employed: string (nullable = true)
 |-- y: string (nullable = true)



In [28]:
# Rename education column value from basic.4y, basic.6y, basic.6y into basic
df_transform2 = df_transform1.withColumn("education",
                                        when(df_transform1.education.endswith('4y'), regexp_replace(df_transform1.education, 'basic.4y', 'basic')) \
                                        .when(df_transform1.education.endswith('6y'), regexp_replace(df_transform1.education, 'basic.6y', 'basic')) \
                                         .when(df_transform1.education.endswith('9y'), regexp_replace(df_transform1.education, 'basic.9y', 'basic')) \
                                         .otherwise(df_transform1.education)
                                        )

In [29]:
df_transform2.show()

+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|              basic|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
| 57|   services| married|        high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.

## Cleanse Null Data

In [32]:
df_transform3 = df_transform2.na.drop("all")

In [33]:
# Seems like there's no null data
df_transform1.count()

41188